In [1]:
%env ROCM_PATH=/opt/rocm-6.2.2
%env AMD_SERIALIZE_KERNEL=3
%env HSA_OVERRIDE_GFX_VERSION=10.3.0
%env GPU_DEVICE_ORDINAL="0"
%env CUDA_VISIBLE_DEVICES="0"
%env PYTORCH_HIP_ALLOC_CONF=expandable_segments:True
import sys
import csv

from datasets import load_dataset
dataset = load_dataset("google/code_x_glue_cc_code_refinement", 'medium')
train = dataset["train"]
val = dataset["validation"]
test = dataset["test"]

env: ROCM_PATH=/opt/rocm-6.2.2
env: AMD_SERIALIZE_KERNEL=3
env: HSA_OVERRIDE_GFX_VERSION=10.3.0
env: GPU_DEVICE_ORDINAL="0"
env: CUDA_VISIBLE_DEVICES="0"
env: PYTORCH_HIP_ALLOC_CONF=expandable_segments:True


In [2]:
from transformers import RobertaTokenizer, RobertaModel

# Load CodeBERT
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
base_model = RobertaModel.from_pretrained("microsoft/codebert-base")

def preprocess_data(data, tokenizer, max_length=512):
    inputs, outputs = [], []
    for entry in data:
        buggy_code = entry.get("buggy")
        fixed_code = entry.get("fixed")
        if buggy_code and fixed_code:
            inputs.append(buggy_code)
            outputs.append(fixed_code)
    input_encodings = tokenizer(inputs, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    output_encodings = tokenizer(outputs, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    labels = output_encodings["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    input_encodings["labels"] = labels
    return input_encodings

2025-01-31 11:55:42.782748: E external/local_xla/xla/stream_executor/plugin_registry.cc:91] Invalid plugin kind specified: FFT
2025-01-31 11:55:43.149498: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-31 11:55:44.145520: E external/local_xla/xla/stream_executor/plugin_registry.cc:91] Invalid plugin kind specified: DNN


In [3]:
import torch
from torch import nn

class Seq2SeqCodeBERT(nn.Module):
    def __init__(self, base_model):
        super(Seq2SeqCodeBERT, self).__init__()
        self.base_model = base_model
        self.lm_head = nn.Linear(base_model.config.hidden_size, base_model.config.vocab_size)
        self.regularization = 1e-5  # L2 regularization factor

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.base_model(input_ids, attention_mask=attention_mask)
        logits = self.lm_head(outputs.last_hidden_state)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss(ignore_index=-100)
            loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
            # Apply L2 regularization
            l2_norm = sum(p.pow(2.0).sum() for p in self.base_model.parameters())
            loss += self.regularization * l2_norm
        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}



In [4]:
import torch
from torch.utils.data import Dataset, DataLoader

class CodeRefinementDataset(Dataset):
    def __init__(self, tokenized_data):
        self.data = tokenized_data

    def __len__(self):
        return len(self.data["input_ids"])

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.data.items()}
train_data = preprocess_data(train, tokenizer)
train_dataset = CodeRefinementDataset(train_data)
train_size = int(0.99907 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [5]:
from transformers import Trainer, TrainingArguments, EvalPrediction
import evaluate
import numpy as np

model = Seq2SeqCodeBERT(base_model)

bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")  # Optional: Requires the METEOR dataset

def compute_metrics(eval_preds: EvalPrediction):
    preds, labels = eval_preds.predictions, eval_preds.label_ids
    pred_ids = np.argmax(preds, axis=-1)

    decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in pred_ids]
    decoded_labels = [
        tokenizer.decode([l for l in label if l != -100], skip_special_tokens=True)
        for label in labels
    ]

    bleu = bleu_metric.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    rouge = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
    meteor = meteor_metric.compute(predictions=decoded_preds, references=decoded_labels)

    return {"bleu": bleu["bleu"], "rougeL": rouge["rougeL"], "meteor": meteor["meteor"]}

training_args = TrainingArguments(
    output_dir="./codebert_seq2seq",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,
    save_steps=1000,
    evaluation_strategy="steps",
    eval_steps=400,
    logging_dir="./logs",
    logging_steps=100,
    weight_decay=1e-5  # Regularization in optimizer
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train model
trainer.train()

Using the latest cached version of the module from /home/ivan/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Sat Jan 25 04:40:33 2025) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.
[nltk_data] Downloading package wordnet to /home/ivan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/ivan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ivan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/home/ivan/.local/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_1361443/2503622106.py:41: FutureWarning: `to

  0%|          | 0/65395 [00:00<?, ?it/s]

/home/ivan/.local/lib/python3.10/site-packages/torch/nn/modules/linear.py:125: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:296.)
  return F.linear(input, self.weight, self.bias)


{'loss': 15.1706, 'grad_norm': 5.536890029907227, 'learning_rate': 4.992354155516477e-05, 'epoch': 0.01}
{'loss': 11.4489, 'grad_norm': 4.532386302947998, 'learning_rate': 4.984708311032954e-05, 'epoch': 0.02}
{'loss': 10.5528, 'grad_norm': 5.624212741851807, 'learning_rate': 4.977062466549431e-05, 'epoch': 0.02}
{'loss': 10.005, 'grad_norm': 2.303966999053955, 'learning_rate': 4.969416622065907e-05, 'epoch': 0.03}


  0%|          | 0/49 [00:00<?, ?it/s]

...

  0%|          | 0/49 [00:00<?, ?it/s]

{'eval_loss': 2.2341341972351074, 'eval_bleu': 0.4789667476042025, 'eval_rougeL': 0.9078740202741424, 'eval_meteor': 0.7599495052651202, 'eval_runtime': 6.8282, 'eval_samples_per_second': 7.176, 'eval_steps_per_second': 7.176, 'epoch': 4.95}
{'loss': 2.2242, 'grad_norm': 4.966334342956543, 'learning_rate': 3.7846930193439866e-07, 'epoch': 4.96}
{'loss': 2.1812, 'grad_norm': 1.9314008951187134, 'learning_rate': 3.020108570991666e-07, 'epoch': 4.97}
{'loss': 2.3406, 'grad_norm': 1.3739429712295532, 'learning_rate': 2.2555241226393456e-07, 'epoch': 4.98}
{'loss': 2.2205, 'grad_norm': 2.0979034900665283, 'learning_rate': 1.490939674287025e-07, 'epoch': 4.99}


  0%|          | 0/49 [00:00<?, ?it/s]

{'eval_loss': 2.23445200920105, 'eval_bleu': 0.47993253913064643, 'eval_rougeL': 0.9082375627318844, 'eval_meteor': 0.7630960246725509, 'eval_runtime': 7.4669, 'eval_samples_per_second': 6.562, 'eval_steps_per_second': 6.562, 'epoch': 4.99}
{'loss': 2.3731, 'grad_norm': 2.86311411857605, 'learning_rate': 7.263552259347045e-08, 'epoch': 4.99}
{'train_runtime': 31382.8829, 'train_samples_per_second': 8.335, 'train_steps_per_second': 2.084, 'train_loss': 2.841464596930716, 'epoch': 5.0}


TrainOutput(global_step=65395, training_loss=2.841464596930716, metrics={'train_runtime': 31382.8829, 'train_samples_per_second': 8.335, 'train_steps_per_second': 2.084, 'total_flos': 0.0, 'train_loss': 2.841464596930716, 'epoch': 5.0})

In [ ]:
def print_sample_predictions(model, tokenizer, dataset, num_samples=5):
    model.eval()  # Set model to evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for i in range(min(num_samples, len(dataset))):
        sample = dataset[i]
        input_ids = sample["input_ids"].unsqueeze(0).to(device)
        attention_mask = sample["attention_mask"].unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs["logits"], dim=-1)

        # Decode input and output text
        input_text = tokenizer.decode(sample["input_ids"], skip_special_tokens=True)
        predicted_text = tokenizer.decode(predictions[0], skip_special_tokens=True)

        print(f"Sample {i+1}:")
        print("Vulnerable Code:")
        print(input_text)
        print("\nPredicted Fixed Code:")
        print(predicted_text)
        print("="*80)

# Call the function to print predictions
print_sample_predictions(model, tokenizer, val_dataset, num_samples=30)


Sample 1:
Vulnerable Code:
public void METHOD_1 ( TYPE_1 item ) { VAR_1 . METHOD_2 ( ) ; for ( int i = 0 ; i < ( VAR_2 . METHOD_3 ( ) ) ; i ++ ) { if ( VAR_2 . METHOD_4 ( i ) . equals ( item ) ) { VAR_1 . METHOD_5 ( i ) . METHOD_6 ( true ) ; } else { VAR_1 . METHOD_5 ( i ) . METHOD_6 ( false ) ; } } }


Predicted Fixed Code:
public void METHOD_1 ( TYPE_1 item ) { VAR_1 . METHOD_2 ( ) ; for ( int i = 0 ; i < ( VAR_2 . METHOD_3 ( ) ) ; i ++ ) { if ( VAR_2 . METHOD_4 ( i ) . equals ( item ) ) { VAR_1 . METHOD_5 ( i ) . METHOD_6 ( true ) ; } else { VAR_1 . METHOD_5 ( i ) . METHOD_6 ( false ) ; } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } } }
Sample 2:
Vulnerable Code:
public boolean METHOD_1 ( final TYPE_1 VAR_1 ) { if ( VAR_2 ) { TYPE_2 VAR_3 = new TYPE_2 ( VAR_4 , VA